In [26]:
import numpy as np
import pandas as pd
import os
import glob
from abc import ABC, ABCMeta, abstractmethod


In [27]:
doc_folder = './doc/'
dsn_cahier_technique = 'dsn_cahier_technique'
excel_extension = '.xlsx'

print(f'Available files in {doc_folder}:')
for file_name in [fn for fn in os.listdir(doc_folder) if not fn.startswith('.')]:
    print('   ', file_name)

dsn_excel = glob.glob(doc_folder+dsn_cahier_technique+'*'+excel_extension)
assert len(dsn_excel) != 0, f'No excel file containing "{dsn_cahier_technique}" found in "{doc_folder}" folder'
assert len(dsn_excel) < 2, f'More than one excel file containing "{dsn_cahier_technique}" found in "{doc_folder}" folder'

dsn_excel = dsn_excel[0]
print('\nExcel file found:')
print('   ', dsn_excel, end='\n\n')

dsn_dfs = pd.read_excel(dsn_excel, sheet_name=None)

def print_data_frames(dfs, verbose=False):
    for (df_name, df) in dfs.items():
        if verbose:
            print(df_name + '_'*(100-len(df_name)))
            print(df.head(5), '\n')
        else:
            print(f"{df_name:15}   [{str(df.columns)}]")
            
print_data_frames(dsn_dfs)


Available files in ./doc/:
    dsn_cahier_technique_p3_2019.1.2.pdf
    dsn_cahier_technique_p3_2019.1.2.xlsx

Excel file found:
    ./doc\dsn_cahier_technique_p3_2019.1.2.xlsx

Standard          [Index(['DSN'], dtype='object')]
Header            [Index(['Element', 'Id', 'Name', 'Description', 'Comment'], dtype='object')]
Data Types        [Index(['Id', 'Nature', 'Regexp', 'Lg Min', 'Lg Max', 'Values'], dtype='object')]
Blocks            [Index(['Id', 'Name', 'Description', 'ParentId', 'lowerBound', 'upperBound'], dtype='object')]
Fields            [Index(['Block Id', 'Id', 'Name', 'Description', 'DataType Id', 'Comment'], dtype='object')]
Messages          [Index(['Name', 'Description', 'Message'], dtype='object')]


In [28]:
class DsnRoot():
    
    def type_(self):
        """
        return the BlockType for block object, RubriqueType for rubrique object
        """
        pass
    
    def conf(self):
        """
        return the BlockConf for block object (except BlockType)
        RubriqueConf for rubrique object
        type object do not have conf
        """
        pass
    
class BlockRoot(DsnRoot, ABC):
    
    @abstractmethod
    def iterate_on_list(self):
        pass

    def __iter__(self):
        return iter(self.iterate_on_list())
    
    def __next__(self):
        return next(self.iterate_on_list())
    
    def __len__(self):
        return len(self.iterate_on_list())
    
    def __getitem__(self, n):
        return self.iterate_on_list()[n]
    
    def __setitem__(self, n, value):
        self.iterate_on_list()[n] = value
        
    def __delitem__(self, n):
        del(self.iterate_on_list()[n])
    
class RubriqueRoot(DsnRoot):
    
    def block(self):
        pass
    
    
class BlockType(BlockRoot):
    ids = {}
    
    def __init__(self, id, name, description='', lower_bound=1, upper_bound=1):
        self.id = id
        self.name = name
        self.full_name = name
        self.description = description
        self.lower_bound = lower_bound
        self.upper_bound = upper_bound
        self.description = description
        self.sub_blocks = []
        self.rubriques = []
        BlockType.ids[id] = self
        
    def __str__(self):
        return f'[{self.id}] {self.name:44} ({str(self.lower_bound):s}, {str(self.upper_bound):s})'
    
    def __repr__(self):
        return f'{type(self).__name__}(id={self.id}, name={self.name}, {self.lower_bound}, {self.upper_bound})' 
    
    def append(self, id, name, description='', lower_bound=1, upper_bound=1):
        self.sub_blocks.append(BlockType(id, name, description, lower_bound, upper_bound))
        
    def iterate_on_list(self):
        return self.sub_blocks
          
    @classmethod
    def append_in_parent(cls, parent_id, id, name, description='', lower_bound=1, upper_bound=1):
        cls.ids[parent_id].append(id, name, description, lower_bound, upper_bound)
    
    @classmethod
    def append_rubrique(cls, block_id, id, name, full_name, description, data_type_id):
        i = name.find('.')
        rubrique_name = name[i+1:]
        name_block = name[:i]
        cls.ids[block_id].name = name_block
        cls.ids[block_id].rubriques.append(RubriqueType(block_id, id, rubrique_name, full_name, description, data_type_id))
        
    def deep_print(self, depth=0, print_rubriques=False):
        print(' '*8*depth + ('' if not depth else'└─ ') + str(self)) # '├─ '
        [print(' '*8*depth + '   > ' + str(r)) for r in self.rubriques if print_rubriques]
        [b.deep_print(depth+1, print_rubriques) for b in self] 
            
class RubriqueType(RubriqueRoot):
    ids = {}
    
    def __init__(self, block_id, id, name, full_name, description, data_type_id):
        self.id = id
        self.name = name
        self.full_name =  full_name
        self.description = description
        self.data_type_id = data_type_id
        self.block_id = block_id
        RubriqueType.ids[block_id+str(id)] = self
    
    def __str__(self):
        return f"{'['+str(self.id)+']':5} {self.name:40}"
    
    def __repr__(self):
        return f'{type(self).__name__}(id={self.id}, name={self.name})' 
            
    def data_type(self):
        return DataType.ids[self.data_type_id]
            
class DataType(DsnRoot):
    ids = {}
    
    def __init__(self, id, nature, regex, lg_min, lg_max, values):
        self.id = id
        self.nature = nature
        self.regex = regex
        self.lg_min, self.lg_max = lg_min, lg_max
        self.values = [tuple(v.split('=')) for v in values.split(';')] if values is not np.nan else []
        DataType.ids[id] = self
        
class RubriqueConf(RubriqueRoot):
    
    def __init__(self, rubrique, is_enabled_by_default=True, use_default_value=False, default_value=''):
        self.rubrique = rubrique
        self.is_enabled_by_default = is_enabled_by_default
        self.use_default_value = use_default_value
        self.default_value = default_value
        
    def __repr__(self):
        return f'{type(self).__name__}(id={self.rubrique.id}, name={self.rubrique.name})'
        
class RubriqueValue(RubriqueRoot):
    
    def __init__(self, rubrique_conf, value='', is_enabled=None):
        self.rubrique_conf = rubrique_conf
        if is_enabled is None:
            self.is_enabled = rubrique_conf.is_enabled_by_default
        else:
            self.is_enabled = bool(is_enabled)
        if value != '':
            self.value = value
        elif rubrique_conf.use_default_value:
            self.value = rubrique_conf.default_value
            
    def __repr__(self):
        return f'{type(self).__name__}(id={self.rubrique_conf.rubrique.id}, name={self.rubrique_conf.rubrique.name})'

class BlockConf(BlockRoot):
    
    def __init__(self, block_type, is_enabled=True):
        self.block_type = block_type
        self.rubriques = [RubriqueConf(r) for r in self.block_type.rubriques]
        self.sub_blocks = [BlockConf(b)  for b in block_type]
        self.is_enabled = is_enabled or self.block_type.lower_bound > 0
        
    def __repr__(self):
        return f'{type(self).__name__}(id={self.block_type.id}, name={self.block_type.name})'
    
    def iterate_on_list(self):
        return self.sub_blocks

class BlockInstance(BlockRoot):

    def __init__(self, block_value):
        self.block_value = block_value
        self.rubriques = [RubriqueValue(r) for r in self.block_value.block_conf.rubriques]
        self.sub_blocks = [BlockValue(b) for b in self.block_value.block_conf]  
            
    def __repr__(self):
        return '{0}(id={1.id}, name={1.name})'.format(type(self).__name__, 
            self.block_value.block_conf.block_type)
              
    def iterate_on_list(self):
        return self.sub_blocks
              
              
class BlockValue(BlockRoot):
    
    def __init__(self, block_conf, number=None):
        self.block_conf = block_conf        
        if number is None:
            number = self.block_conf.block_type.lower_bound
            if not number and self.block_conf.block_type.upper_bound == '*':
                number = 2
                    
            
        self.instances = [BlockInstance(self) for i in range(number)]
  
    def __repr__(self):
        return '{0}(id={1.id}, name={1.name}, number={2})'.format(type(self).__name__,
                self.block_conf.block_type, len(self))
    
    def iterate_on_list(self):
        return self.instances
    

        


In [29]:
root = BlockType(id='DSN', name='DSN_Root', )
root.append('S10.G00.00', 'Envoi')
root.append('S20.G00.05', 'Déclaration')
root.append('S90.G00.90', "Total de l'envoi") 

df_blocks = dsn_dfs['Blocks']
df_rubriques = dsn_dfs['Fields']
df_data_type = dsn_dfs['Data Types']

fd_blocks_index_mapping = {'Id':'id', 'Name':'name', 'ParentId':'parent_id', 'lowerBound':'lower_bound', 
                           'upperBound':'upper_bound', 'Description':'description'}
for index, row in df_blocks.iterrows():
    args = {fd_blocks_index_mapping[key]:value for key, value in row.items()}
    BlockType.append_in_parent(**args)
    
fd_rubriques_index_mapping = {'Block Id':'block_id', 'Id':'id', 'Name':'full_name', 'Description':'description',
                              'DataType Id':'data_type_id', 'Comment':'name'}
for index, row in df_rubriques.iterrows():
    args = {fd_rubriques_index_mapping[key]:value for key, value in row.items()}
    BlockType.append_rubrique(**args)
    
fd_data_type_index_mapping = {'Id':'id', 'Nature':'nature', 'Regexp':'regex', 
                              'Lg Min':'lg_min', 'Lg Max':'lg_max', 'Values':'values'}
for index, row in df_data_type.iterrows():
    args = {fd_data_type_index_mapping[key]:value for key, value in row.items()}
    DataType(**args)


In [30]:
print('Block name max lenght')
max_lenght_block = max(len(i.name) for i in BlockType.ids.values())
[print(i, max_lenght_block) for i in BlockType.ids.values() if len(i.name) == max_lenght_block]
print('\nRubrique name max lenght')
max_lenght_rub = max(len(i.name) for i in RubriqueType.ids.values())
[print(i, max_lenght_rub) for i in RubriqueType.ids.values() if len(i.name) == max_lenght_rub]
print('\n\n')
root.deep_print(print_rubriques=False)

Block name max lenght
[S21.G00.16] ChangementsDestinataireAdhesionPrevoyance    (0, *) 41

Rubrique name max lenght
[63]  FPIndiceBrutOrigineAncienSalarieEmployeurPublic 47
[65]  FPMaintienTraitementOrigineContractuelTitulaire 47
[42]  FPMaintienTraitementOrigineContractuelTitulaire 47



[DSN] DSN_Root                                     (1, 1)
        └─ [S10.G00.00] Envoi                                        (1, 1)
                └─ [S10.G00.01] Emetteur                                     (1, 1)
                └─ [S10.G00.02] Contact                                      (1, 1)
        └─ [S20.G00.05] Declaration                                  (1, 1)
                └─ [S20.G00.07] ContactDeclare                               (0, *)
                └─ [S20.G00.08] IdentifiantOrganismeDestinataireNeant        (0, *)
                └─ [S21.G00.06] Entreprise                                   (1, 1)
                        └─ [S21.G00.11] Etablissement                           

In [31]:
bcr = BlockConf(root)

def printBConf(bc):
    print(bc)
    for b in bc:
        printBConf(b)
        
bvr = BlockValue(bcr)

def printBValue(bc):
    print(bc)
    for i in bc:
        for sb in i:
            printBValue(sb)
#printBValue(bvr)
bir = bvr[0]

def printBInstance(bi):
    print(bi)
    for bv in bi:
        for sbi in bv:
                printBInstance(sbi)
            
printBInstance(bir)

BlockInstance(id=DSN, name=DSN_Root)
BlockInstance(id=S10.G00.00, name=Envoi)
BlockInstance(id=S10.G00.01, name=Emetteur)
BlockInstance(id=S10.G00.02, name=Contact)
BlockInstance(id=S20.G00.05, name=Declaration)
BlockInstance(id=S20.G00.07, name=ContactDeclare)
BlockInstance(id=S20.G00.07, name=ContactDeclare)
BlockInstance(id=S20.G00.08, name=IdentifiantOrganismeDestinataireNeant)
BlockInstance(id=S20.G00.08, name=IdentifiantOrganismeDestinataireNeant)
BlockInstance(id=S21.G00.06, name=Entreprise)
BlockInstance(id=S21.G00.11, name=Etablissement)
BlockInstance(id=S21.G00.15, name=PrevoyanceAdhesion)
BlockInstance(id=S21.G00.16, name=ChangementsDestinataireAdhesionPrevoyance)
BlockInstance(id=S21.G00.16, name=ChangementsDestinataireAdhesionPrevoyance)
BlockInstance(id=S21.G00.15, name=PrevoyanceAdhesion)
BlockInstance(id=S21.G00.16, name=ChangementsDestinataireAdhesionPrevoyance)
BlockInstance(id=S21.G00.16, name=ChangementsDestinataireAdhesionPrevoyance)
BlockInstance(id=S21.G00.82, na

In [32]:
import sys
from PyQt5.QtWidgets import *
import PyQt5.QtCore as Qt

class BlockConfWidget(QWidget):
    
    def __init__(self, master_layout, block, depth=0):
        self.subs = []
        
        super().__init__()
        self.block = block
        self.master_layout = master_layout 
        
        # Create frame
        self.frame = QFrame()
        self.frame.setFrameStyle(QFrame.Panel | QFrame.Raised)
        self.frame.setLineWidth(2)
        
        # Create front check box (disable config block and children if unchecked)
        self.front_check_box = QCheckBox()
        # Can not be unchecked if minumim block number is 1 or greater
        if block.block_type.lower_bound > 0: 
            self.front_check_box.setCheckState(Qt.Qt.PartiallyChecked)
            self.front_check_box.setEnabled(False)
        else:
            self.front_check_box.setChecked(block.is_enabled)
        self.front_check_box.stateChanged.connect(self.on_state_change)
        
        # Create main label of the frame
        self.main_label = QLabel()
        self.main_label.setText(repr(block))
        self.main_label.setMaximumWidth (400)
        
        # Layout for the frame
        self.frameLayout = QHBoxLayout()
        self.frameLayout.addWidget(self.front_check_box)
        self.frameLayout.addWidget(self.main_label)
        self.frame.setLayout(self.frameLayout)
        self.frameLayout.setContentsMargins(5, 5, 5, 5)
        
        # Layout of the widget
        self.layout = QHBoxLayout()
        self.layout.setContentsMargins(0, 0, 0, 0)
        self.layout.addSpacing(depth*40)
        self.layout.addWidget(self.frame) 
        self.layout.addStretch()
        self.setLayout(self.layout)
        self.master_layout.addWidget(self)
        for sub_block in block: 
                self.subs.append(BlockConfWidget(master_layout, sub_block, depth+1))
        if not self.block.is_enabled:
            self.set_subs_visible(False)
        
    def on_state_change(self, state):
        self.block.is_enabled = bool(state)
        self.set_subs_visible(bool(state))

    def set_subs_visible(self, state):
        for sb in self.subs:  
            sb.setVisible(state)
            sb.set_subs_visible(state)


In [33]:
app=QApplication.instance() # checks if QApplication already exists 
if not app: # create QApplication if it doesnt exist 
     app = QApplication(sys.argv)
                 
mainWindow = QMainWindow()
mainWindow.resize(1200, 800)
sa = QScrollArea()
#sa.setGeometry(20, 20, 1160, 760)
sa.setWidgetResizable(True)
sa.setFrameStyle(QFrame.NoFrame)
mainWindow.setCentralWidget(sa)
#saLayout = QHBoxLayout()
#saLayout.addWidget(sa)
#mainWindow.setLayout(saLayout)

w = QWidget()
#w.resize(2600, 800)
sa.setWidget(w)

wLayout = QVBoxLayout()
#wLayout.setContentsMargins(0, 0, 0, 0)



b = BlockConfWidget(wLayout, bcr, 0)
#wLayout.addStretch()
w.setLayout(wLayout)
mainWindow.show()
app.exec_()


0

In [34]:
int?